In [4]:
!pip install -qU langchain

In [1]:
import json
import sagemaker
import boto3
import re
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

# connect to SageMaker
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
ENDPOINT_NAME = 'sm-llm-aws'
runtime = boto3.client('runtime.sagemaker')

In [5]:
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from typing import Dict

In [75]:
client = boto3.client("sagemaker-runtime")

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs" : f"{prompt}", "parameters" : {**model_kwargs}})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        regex = "### Respond\":\n\"(.+)[\'\"]"
        response = re.search(regex, response_json[0]["generated_text"])[1]
        return response
        #return response_json[0]["generated_text"]

content_handler = ContentHandler()

In [76]:
prompt_template = """You are a Spanish teacher. Your respond in Spanish.

### Input:\n{context}
### Respond":"""

In [77]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context"]
)

In [78]:
PROMPT

PromptTemplate(input_variables=['context'], template='You are a Spanish teacher. Your respond in Spanish.\n\n### Input:\n{context}\n### Respond":')

In [79]:
chain = LLMChain(
    llm=SagemakerEndpoint(
        endpoint_name=ENDPOINT_NAME,
        client=client,
        model_kwargs={"temperature": 1e-3},
        #model_kwargs=parameters,
        content_handler=content_handler,
    ),
    prompt=PROMPT,
)

In [80]:
# user input (with mistake) and next question

# pair 1
# Assistant: Hola, ¿cómo estás? (Hello, how are you?)
#user_input = "Ho0ola."
# user_input = "Estoy bienes, gracias."
# next_question = "¿Estás libre hoy?"

# pair 2
# user_input = "Sí, tengo algo de tiempos hoy."
# next_question = "¿Quieres ir de compras conmigo?"

# pair 3
# user_input = "Sí, necesito comprar uno chaqueta."
# next_question = "¿A qué hora te gustaría ir?"

# pair 4
user_input = "A los diez."
next_question = "Vale, nos vemos luego."

# pair 5
# user_input = "Hastas luego."
# next_question = "adiós!"

In [81]:
nogec_prompt = f"rephrase '{next_question}' in Spanish and nothing else:"
gec_prompt = f"'{user_input}' has grammetical error. Return the correction and nothing else:"

In [82]:
%%time

chain.run(context=nogec_prompt, return_only_outputs = True)

CPU times: user 21.3 ms, sys: 2.7 ms, total: 24 ms
Wall time: 745 ms


'Claro, hasta luego." o "Por supuesto, hasta luego.'

In [83]:
%%time

scaffold_output = chain.run(
    "Response with 'Veo. quieres decir " + chain.run(gec_prompt) + "' and nothing else:"
)

scaffold_output

TypeError: 'NoneType' object is not subscriptable